<a href="https://colab.research.google.com/github/hajerhajjajifahem24/Repo-2017/blob/master/lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
import string
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

import nltk
nltk.download('punkt')
import nltk
nltk.download('stopwords')
import nltk
nltk.download('wordnet')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [ ]:

from google.colab import files
uploaded = files.upload()

In [ ]:

import io
dataset = pd.read_csv(io.BytesIO(uploaded['googleplaystore_user_reviews.csv']))

In [ ]:
dataset.head()


In [ ]:
dataset=dataset.dropna()

In [ ]:
dataset

In [ ]:
def preprocess_sentence_text(sentence):
    sentence=sentence.lower()
    # Remove urls
    sentence = re.sub(r"http\S+|www\S+|https\S+", '', sentence, flags=re.MULTILINE)
    # Remove user @ references and '#' from sentence
    sentence = re.sub(r'\@\w+|\#','', sentence)
    # Remove punctuations
    sentence = sentence.translate(str.maketrans('', '', string.punctuation))
    # Remove stopwords
    sentence_tokens = word_tokenize(sentence)
    filtered_words = [w for w in sentence_tokens if not w in stop_words]
    
    ps = PorterStemmer()
    stemmed_words = [ps.stem(w) for w in filtered_words]
    lemmatizer = WordNetLemmatizer()
    lemma_words = [lemmatizer.lemmatize(w, pos='v') for w in stemmed_words]
    
    return " ".join( stemmed_words)

In [ ]:
test="I have been loving this phone"

In [ ]:
test=preprocess_sentence_text(test)

In [ ]:
test

In [ ]:
dataset.Sentiment = [4 if i=="Positive" else 0 if i== "Negative" else 2 for i in dataset.Sentiment]

In [ ]:
dataset.Translated_Review = dataset['Translated_Review'].apply(preprocess_sentence_text)

In [ ]:
dataset=dataset.dropna()

In [ ]:
sentences = dataset['Translated_Review'].tolist()
labels = dataset['Sentiment'].tolist()

In [ ]:
sentences

In [ ]:

print(sentences[2])
print(labels[2])

In [ ]:
training_size = int(len(sentences) * 0.8)

training_sentences = sentences[0: training_size]
testing_senteces = sentences[: training_size]
training_labels = labels[0: training_size]
testing_labels = labels[: training_size]

# Put labels into list to use later:

training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [ ]:

vocab_size = 1000
embedding_dim = 16
max_length = 280
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

In [ ]:

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index



In [ ]:
len(word_index)

In [ ]:
training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_senteces)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [ ]:
training_sequences

In [ ]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length))
model.add(tf.keras.layers.Bidirectional(
    tf.keras.layers.LSTM(embedding_dim,
                         return_sequences=True)
))
model.add(tf.keras.layers.Dense(6, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [ ]:

model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(0.01),
              metrics=['accuracy'])

In [ ]:
callbacks = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto',
    baseline=None, restore_best_weights=False
)
num_epochs=10
modelo = model.fit(training_padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final),
          callbacks=[callbacks])

In [ ]:

plt.plot(modelo.history['accuracy'])
plt.xlabel('Epochs')
plt.ylabel('Accuracy')

In [ ]:
plt.plot(modelo.history['loss'])
plt.xlabel('Epochs')
plt.ylabel('Accuracy')

In [ ]:

# Use the model to predict a review   
fake_reviews = ['I love this phone']

print(fake_reviews) 

# Create the sequences
padding_type='post'
sample_sequences = tokenizer.texts_to_sequences(fake_reviews)
fakes_padded = pad_sequences(sample_sequences, padding=padding_type, maxlen=max_length)           

classes = model.predict(fakes_padded)

In [ ]:
print(classes)

In [ ]:
y = np.argmax(classes, axis=1)

In [ ]:
print(y)